# Dashboard Data

This Jupyter notebook is the data preparation script for the data to be displayed in the dashboard. As the integration of MongoDB in Google Data Studio required multiple intermediate layers using a Data Warehouse such as Big Query. As this was not needed for the project, it was decided to export the processed csv data as the data was not going to be modified dynamically. 


This script requires the following packages:

In [1]:
import sys
import pandas as pd
import numpy as np
from pymongo import MongoClient
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import config
import datetime


## Fetching the Data

Fetching the raw sensor data from mongodb using secure connection. 

In [16]:
df = pd.read_csv("Export_Data/data.csv")

In [17]:
df.head()

,Unnamed: 0,occupancy,bin1,bin2,H2S,temperature,humidity,timestamp,date,time
0,0,5,27.81,0.0,915,26.7,40.0,1638867600,2021-12-07,09:00:00
1,1,6,29.48,0.0,843,26.3,30.0,1638868200,2021-12-07,09:10:00
2,2,6,37.06,0.0,826,27.6,26.0,1638868800,2021-12-07,09:20:00
3,3,6,-1.00,0.0,837,28.0,25.0,1638869400,2021-12-07,09:30:00
4,4,6,-1.00,0.0,834,28.0,26.0,1638870000,2021-12-07,09:40:00


In [18]:
df = df.drop("Unnamed: 0", axis=1)

In [19]:
df

,occupancy,bin1,bin2,H2S,temperature,humidity,timestamp,date,time
0,5,27.81,0.0,915,26.7,40.0,1638867600,2021-12-07,09:00:00
1,6,29.48,0.0,843,26.3,30.0,1638868200,2021-12-07,09:10:00
2,6,37.06,0.0,826,27.6,26.0,1638868800,2021-12-07,09:20:00
3,6,-1.00,0.0,837,28.0,25.0,1638869400,2021-12-07,09:30:00
4,6,-1.00,0.0,834,28.0,26.0,1638870000,2021-12-07,09:40:00
...,...,...,...,...,...,...,...,...,...
1362,0,33.63,-1.0,-1,-1.0,-1.0,1639725600,2021-12-17,07:20:00
1363,0,29.49,-1.0,-1,-1.0,-1.0,1639726200,2021-12-17,07:30:00
1364,0,29.49,-1.0,-1,-1.0,-1.0,1639726800,2021-12-17,07:40:00
1365,0,29.49,-1.0,-1,-1.0,-1.0,1639727400,2021-12-17,07:50:00


## Daily H2S values

In [6]:
dailyH2S = pd.DataFrame()

for i in range(141):
    dailyH2S.loc[i,"time"] = df.loc[89+i, "timestamp"]
    dailyH2S.loc[i,"Dec08"] = df.loc[89+i, "H2S"]
    dailyH2S.loc[i,"Dec09"] = df.loc[89+(141*1)+i, "H2S"]
    dailyH2S.loc[i,"Dec10"] = df.loc[89+(141*2)+i, "H2S"]
    dailyH2S.loc[i,"Dec11"] = df.loc[89+(141*3)+i, "H2S"]
    dailyH2S.loc[i,"Dec12"] = df.loc[89+(141*4)+i, "H2S"]
    dailyH2S.loc[i,"Dec13"] = df.loc[89+(141*5)+i, "H2S"]
    dailyH2S.loc[i,"Dec14"] = df.loc[89+(141*6)+i, "H2S"]
    
for j in range(131):
    dailyH2S.loc[j,"Dec15"] = df.loc[1094+j, "H2S"]
for k in range(131, 141):
    dailyH2S.loc[k,"Dec15"] = -1
for m in range(48):
    dailyH2S.loc[m,"Dec16"] = -1
for l in range(len(df[df['date'] == datetime.date(2021, 12, 16)]["H2S"])):
    dailyH2S.loc[48+l,"Dec16"] = df.loc[1225+l, "H2S"]



In [7]:
dailyH2S.tail()

,time,Dec08,Dec09,Dec10,Dec11,Dec12,Dec13,Dec14,Dec15,Dec16
136,1.639005e+09,880.0,935.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN
137,1.639006e+09,850.0,880.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN
138,1.639006e+09,867.0,891.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN
139,1.639007e+09,883.0,884.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN
140,1.639007e+09,857.0,917.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN


In [8]:
dailyH2S = dailyH2S.replace(-1,0)

In [9]:
dailyH2S['hour_minute'] = pd.to_datetime(dailyH2S['time'], unit='s').dt.time


In [10]:
df[df['date'] == datetime.date(2021, 12, 16)]["H2S"]


Series([], Name: H2S, dtype: int64)

In [11]:
dailyH2S.head()

,time,Dec08,Dec09,Dec10,Dec11,Dec12,Dec13,Dec14,Dec15,Dec16,hour_minute
0,1.638922e+09,0.0,858.0,891.0,0.0,0.0,0.0,0.0,0.0,0.0,00:00:00
1,1.638922e+09,0.0,850.0,842.0,0.0,0.0,0.0,0.0,0.0,0.0,00:10:00
2,1.638923e+09,0.0,839.0,866.0,0.0,0.0,0.0,0.0,0.0,0.0,00:20:00
3,1.638923e+09,0.0,838.0,908.0,0.0,0.0,0.0,0.0,0.0,0.0,00:30:00
4,1.638924e+09,0.0,816.0,878.0,0.0,0.0,0.0,0.0,0.0,0.0,00:40:00


## Daily Bin Max Volume

Collating the largest values from each day to identify which day had more overall trash thrown away more visually

In [57]:
dailyBin = pd.DataFrame()

# Indexing issue when automated using a for loop - manual alternative did the job

dailyBin.loc[0,"min1"] = df[(df["date"] == str(datetime.date(2021, 12, 8))) & (df["bin1"]>0)]["bin1"].min()
dailyBin.loc[1,"min1"] = df[(df["date"] == str(datetime.date(2021, 12, 9))) & (df["bin1"]>0)]["bin1"].min()
dailyBin.loc[2,"min1"] = df[(df["date"] == str(datetime.date(2021, 12, 10))) & (df["bin1"]>0)]["bin1"].min()
dailyBin.loc[3,"min1"] = df[(df["date"] == str(datetime.date(2021, 12, 11))) & (df["bin1"]>0)]["bin1"].min()
dailyBin.loc[4,"min1"] = df[(df["date"] == str(datetime.date(2021, 12, 12))) & (df["bin1"]>0)]["bin1"].min()
dailyBin.loc[5,"min1"] = df[(df["date"] == str(datetime.date(2021, 12, 13))) & (df["bin1"]>0)]["bin1"].min()
dailyBin.loc[6,"min1"] = df[(df["date"] == str(datetime.date(2021, 12, 14))) & (df["bin1"]>0)]["bin1"].min()
dailyBin.loc[7,"min1"] = df[(df["date"] == str(datetime.date(2021, 12, 15))) & (df["bin1"]>0)]["bin1"].min()
dailyBin.loc[8,"min1"] = df[(df["date"] == str(datetime.date(2021, 12, 16))) & (df["bin1"]>0)]["bin1"].min()
dailyBin.loc[0+9,"min1"] = df[df["date"] == str(datetime.date(2021, 12, 8))]["bin1"].max()
dailyBin.loc[1+9,"min1"] = df[df["date"] == str(datetime.date(2021, 12, 9))]["bin1"].max()
dailyBin.loc[2+9,"min1"] = df[df["date"] == str(datetime.date(2021, 12, 10))]["bin1"].max()
dailyBin.loc[3+9,"min1"] = df[df["date"] == str(datetime.date(2021, 12, 11))]["bin1"].max()
dailyBin.loc[4+9,"min1"] = df[df["date"] == str(datetime.date(2021, 12, 12))]["bin1"].max()
dailyBin.loc[5+9,"min1"] = df[df["date"] == str(datetime.date(2021, 12, 13))]["bin1"].max()
dailyBin.loc[6+9,"min1"] = df[df["date"] == str(datetime.date(2021, 12, 14))]["bin1"].max()
dailyBin.loc[7+9,"min1"] = df[df["date"] == str(datetime.date(2021, 12, 15))]["bin1"].max()
dailyBin.loc[8+9,"min1"] = df[df["date"] == str(datetime.date(2021, 12, 16))]["bin1"].max()

dailyBin.loc[0,"min2"] = df[(df["date"] == str(datetime.date(2021, 12, 8))) & (df["bin2"]>0)]["bin2"].min()
dailyBin.loc[1,"min2"] = df[(df["date"] == str(datetime.date(2021, 12, 9))) & (df["bin2"]>0)]["bin2"].min()
dailyBin.loc[2,"min2"] = df[(df["date"] == str(datetime.date(2021, 12, 10))) & (df["bin2"]>0)]["bin2"].min()
dailyBin.loc[3,"min2"] = df[(df["date"] == str(datetime.date(2021, 12, 11))) & (df["bin2"]>0)]["bin2"].min()
dailyBin.loc[4,"min2"] = df[(df["date"] == str(datetime.date(2021, 12, 12))) & (df["bin2"]>0)]["bin2"].min()
dailyBin.loc[5,"min2"] = df[(df["date"] == str(datetime.date(2021, 12, 13))) & (df["bin2"]>0)]["bin2"].min()
dailyBin.loc[6,"min2"] = df[(df["date"] == str(datetime.date(2021, 12, 14))) & (df["bin2"]>0)]["bin2"].min()
dailyBin.loc[7,"min2"] = df[(df["date"] == str(datetime.date(2021, 12, 15))) & (df["bin2"]>0)]["bin2"].min()
dailyBin.loc[8,"min2"] = df[(df["date"] == str(datetime.date(2021, 12, 16))) & (df["bin2"]>0)]["bin2"].min()
dailyBin.loc[0+9,"min2"] = df[df["date"] == str(datetime.date(2021, 12, 8))]["bin2"].max()
dailyBin.loc[1+9,"min2"] = df[df["date"] == str(datetime.date(2021, 12, 9))]["bin2"].max()
dailyBin.loc[2+9,"min2"] = df[df["date"] == str(datetime.date(2021, 12, 10))]["bin2"].max()
dailyBin.loc[3+9,"min2"] = df[df["date"] == str(datetime.date(2021, 12, 11))]["bin2"].max()
dailyBin.loc[4+9,"min2"] = df[df["date"] == str(datetime.date(2021, 12, 12))]["bin2"].max()
dailyBin.loc[5+9,"min2"] = df[df["date"] == str(datetime.date(2021, 12, 13))]["bin2"].max()
dailyBin.loc[6+9,"min2"] = df[df["date"] == str(datetime.date(2021, 12, 14))]["bin2"].max()
dailyBin.loc[7+9,"min2"] = df[df["date"] == str(datetime.date(2021, 12, 15))]["bin2"].max()
dailyBin.loc[8+9,"min2"] = df[df["date"] == str(datetime.date(2021, 12, 16))]["bin2"].max()


In [58]:
dailyBin

,min1,min2
0,11.42,17.37
1,7.16,8.36
2,8.92,11.01
3,21.75,11.01
4,11.55,10.92
5,16.02,8.95
6,22.80,12.78
7,22.62,14.53
8,24.37,29.35
9,49.91,48.27


In [59]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler((0,100))
# scaler = StandardScaler()
dailyBin[["min1","min2"]] = scaler.fit_transform(-dailyBin[["min1","min2"]])


# col_names = ["min1","min2"]
# features = dailyBin[col_names]

# # Use scaler of choice; here Standard scaler is used
# scaler = MinMaxScaler((0,20)).fit(-features.values)
# features = scaler.transform(features.values)

# dailyBin[col_names] = features

In [60]:
dailyBin

,min1,min2
0,90.116009,83.461821
1,100.000000,100.000000
2,95.916473,95.135830
3,66.148492,95.135830
4,89.814385,95.301028
5,79.443155,98.917034
6,63.712297,91.886931
7,64.129930,88.674743
8,60.069606,61.472100
9,0.812065,26.743759


In [61]:
dailyBin["min1"] = dailyBin["min1"].astype(int)
dailyBin["min2"] = dailyBin["min2"].astype(int)

In [62]:
dailyBin= dailyBin[dailyBin.index < 9] 


In [63]:
dailyBin = dailyBin.rename(columns={"min1": "Bin1", "min2": "Bin2"})

In [64]:
import time 
for i in range(len(dailyBin)):
    dailyBin.loc[i,"date"] = datetime.date(2021, 12, 8+i)
    dailyBin.loc[i,"timestamp"] = time.mktime(datetime.date(2021, 12, 8+i).timetuple())

In [65]:
dailyBin

,Bin1,Bin2,date,timestamp
0,90,83,2021-12-08,1.638940e+09
1,100,100,2021-12-09,1.639026e+09
2,95,95,2021-12-10,1.639112e+09
3,66,95,2021-12-11,1.639199e+09
4,89,95,2021-12-12,1.639285e+09
5,79,98,2021-12-13,1.639372e+09
6,63,91,2021-12-14,1.639458e+09
7,64,88,2021-12-15,1.639544e+09
8,60,61,2021-12-16,1.639631e+09


## Week Data

Exporting the data for the weekly 

In [66]:
weekData = pd.DataFrame()
weekData["occupancy"] = np.nan
_index = 0

for i in range(9):
    for j in range(12):
        for k in range(2):
            location = df[(df["date"] == datetime.date(2021, 12, 8+i)) & (df["time"] == datetime.time(2*j,0,0))]
            try:
                weekData.loc[_index, "occupancy"] = location["occupancy"].item()
            except:
                weekData.loc[_index, "occupancy"] = 0
            try:
                weekData.loc[_index, "humidity"] = location["humidity"].item()
            except:
                weekData.loc[_index, "humidity"] = 0
            try:
                weekData.loc[_index, "temperature"] = location["temperature"].item()
            except:
                weekData.loc[_index, "temperature"] = 0
            try:
                weekData.loc[_index, "date"] = location["date"].item()
            except:
                weekData.loc[_index, "date"] = 0
            try:
                weekData.loc[_index, "time"] = location["time"].item()
            except:
                weekData.loc[_index, "time"] = 0
            try:
                weekData.loc[_index, "timestamp"] = location["timestamp"].item()
            except:
                weekData.loc[_index, "timestamp"] = 0
            # weekData.loc[_index,"date"] = location["date"].item()
            # weekData.loc[_index,"time"] = location["time"].item()
            # weekData.loc[_index,"timestamp"] = location["timestamp"].item()
            _index += 1

In [67]:
weekData = weekData[weekData.date != 0]


In [340]:
dailyH2S.to_csv("Export_Data/dailyH2S.csv", index = True, header=True)
dailyBin.to_csv("Export_Data/dailyBin.csv", index = True, header=True)
weekData.to_csv("Export_Data/weekData.csv", index = True, header=True)